In [108]:
import sklearn 
import neurolab as nl
import numpy as np
import scipy 
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation as cv
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split

### LogReg

In [109]:
df_train = pd.read_csv('OnlineNewsPopularity.csv')
df_train = df_train[:32000]
df_train.shape
#df_train["Popularity"]=0

(32000, 64)

In [110]:
df_tr = df_train[:25000]
del df_tr["url"]
df_tr_output = df_train["Popularity"][:25000]
#del df_tr["Popularity"]
del df_tr[" shares"]
print df_tr.shape

(25000, 62)


In [111]:
df_te = df_train[25000:]
del df_te["url"]
df_te_output = df_train["Popularity"][25000:]
#del df_te["Popularity"]
del df_te[" shares"]
print df_te.shape


(7000, 62)


In [112]:
def take_log(df):
    df_log = np.log(df).replace([np.inf, -np.inf], np.nan).fillna(0)
    return df_log



In [113]:
def zscore(data):
   zmean=data.mean()
   zstd=data
   zscore_input = (data-data.mean())/data.std(ddof=0)
   return zscore_input

In [121]:
# df_tr["multiple"] = zscore(df_tr["multiple"])
df_tr["multiple2"] = take_log(df_tr["multiple2"])
df_tr[" timedelta"] = take_log(df_tr[" timedelta"])
df_tr[" self_reference_min_shares"] = take_log(df_tr[" self_reference_min_shares"])
df_tr[" kw_avg_min"] = take_log(df_tr[" kw_avg_min"])
df_tr[" kw_avg_max"] = take_log(df_tr[" kw_avg_max"])
df_tr[" kw_max_avg"] = take_log(df_tr[" kw_max_avg"])
df_tr[" kw_avg_avg"] = take_log(df_tr[" kw_avg_avg"])

df_te[" timedelta"] = take_log(df_te[" timedelta"])
df_te[" self_reference_min_shares"] = take_log(df_te[" self_reference_min_shares"])
df_te[" kw_avg_min"] = take_log(df_te[" kw_avg_min"])
df_te[" kw_avg_max"] = take_log(df_te[" kw_avg_max"])
df_te[" kw_max_avg"] = take_log(df_te[" kw_max_avg"])
df_te[" kw_avg_avg"] = take_log(df_te[" kw_avg_avg"])

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

In [122]:
pred = df_tr.columns
predictors = []
for each in pred:
    if each !=  "Popularity":
        predictors.append(each)
    else:
        continue

In [123]:

#predictors = [" n_unique_tokens", " n_non_stop_words", " data_channel_is_entertainment", " is_weekend", " LDA_00", " LDA_02", " global_subjectivity", " rate_positive_words", " rate_negative_words", " data_channel_is_bus"]--93, 61.04, 60.1
predictors = [#' weekday_is_monday'," timedelta",
 #' weekday_is_tuesday',
 #' weekday_is_wednesday',
 #' weekday_is_thursday',
 #' weekday_is_friday',
    'multiple2',
 ' weekday_is_saturday',
 ' weekday_is_sunday',' global_subjectivity',' n_non_stop_unique_tokens',
 #' global_sentiment_polarity',
 ' global_rate_positive_words',' num_hrefs',
   " n_non_stop_words",' max_positive_polarity',' max_negative_polarity',' min_positive_polarity',
 ' global_rate_negative_words',' average_token_length', " n_unique_tokens"," data_channel_is_entertainment"," rate_positive_words", " rate_negative_words"," is_weekend", " kw_avg_min", " kw_avg_max", " kw_max_avg",
            " kw_avg_avg"," data_channel_is_bus", " self_reference_min_shares", " LDA_00", " LDA_01", " LDA_04"," avg_positive_polarity"," avg_negative_polarity"]#--85, 0.6543461538461538, 0.6435063031369101

##predictors = [" timedelta", " n_unique_tokens"," data_channel_is_entertainment"," rate_positive_words", " rate_negative_words"," is_weekend", " kw_avg_min", " kw_avg_max", " kw_max_avg",
#            " kw_avg_avg"," data_channel_is_bus", " self_reference_min_shares", " LDA_00", " LDA_01", " LDA_04"]#log--86, 0.6448076923076923, 0.64724420990911757

#predictors = [" timedelta", " n_unique_tokens"," data_channel_is_entertainment"," rate_positive_words", " rate_negative_words"," is_weekend", " kw_avg_min", " kw_avg_max", " kw_max_avg",
#            " kw_avg_avg"," data_channel_is_bus", " self_reference_min_shares", " LDA_00", " LDA_01", " LDA_04"," avg_positive_polarity"," avg_negative_polarity"]---94, 0.654076923076923,0.64167399589563176

#predictors = [' timedelta', ' n_tokens_content', ' n_unique_tokens', ' average_token_length', ' kw_avg_min', ' kw_avg_max', ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares', ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_04', ' global_subjectivity', ' avg_positive_polarity']--90, 0.65111538461538476,0.64028144239226037
#predictors = [' timedelta', ' n_tokens_content', ' n_unique_tokens', ' average_token_length', ' kw_avg_min', ' kw_avg_max', ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares', ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_04', ' global_subjectivity', ' avg_positive_polarity'," rate_positive_words", " rate_negative_words"]#--92, 0.64849999999999997,0.63522427440633245

#predictors = [" timedelta", " n_unique_tokens"," data_channel_is_entertainment"," rate_positive_words", " rate_negative_words"," is_weekend", " kw_avg_min", " kw_avg_max", " kw_max_avg",
#            " kw_avg_avg"," data_channel_is_bus", " self_reference_min_shares", " LDA_00", " LDA_01", " LDA_04"]#after taking log of large valued columns--64.4 

print len(predictors)

df_tr[predictors].shape

29


(25000, 29)

In [124]:
predictors

['multiple2',
 ' weekday_is_saturday',
 ' weekday_is_sunday',
 ' global_subjectivity',
 ' n_non_stop_unique_tokens',
 ' global_rate_positive_words',
 ' num_hrefs',
 ' n_non_stop_words',
 ' max_positive_polarity',
 ' max_negative_polarity',
 ' min_positive_polarity',
 ' global_rate_negative_words',
 ' average_token_length',
 ' n_unique_tokens',
 ' data_channel_is_entertainment',
 ' rate_positive_words',
 ' rate_negative_words',
 ' is_weekend',
 ' kw_avg_min',
 ' kw_avg_max',
 ' kw_max_avg',
 ' kw_avg_avg',
 ' data_channel_is_bus',
 ' self_reference_min_shares',
 ' LDA_00',
 ' LDA_01',
 ' LDA_04',
 ' avg_positive_polarity',
 ' avg_negative_polarity']

In [118]:
#Applying method
max_score = 0
best_n = 0.
for n in range(85,101):
    rfc_scr = 0.
    rfc = RandomForestClassifier(n_estimators=n)
    for train, test in KFold(len(df_tr), n_folds=10, shuffle=True):
        rfc.fit(df_tr[predictors].T[train].T, df_tr["Popularity"].T[train].T)
        rfc_scr += rfc.score(df_tr[predictors].T[test].T, df_tr["Popularity"].T[test].T)/10
    if rfc_scr > max_score:
        max_score = rfc_scr
        best_n = n 
    print n
print(best_n, max_score)
rfc = RandomForestClassifier(best_n)


KeyboardInterrupt: 

In [125]:
rfc = RandomForestClassifier(88)

In [126]:
rfc.fit(df_tr[predictors], df_tr["Popularity"])
rfc.score(df_te[predictors], df_te["Popularity"])

0.64685714285714291